In [ ]:
import torch

# Verify if Cuda is available to use GPU
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible en tu sistema.")
else:
    print("CUDA (GPU) no está disponible en tu sistema.")

# AI by reinforcement learning 
---
This artificial intelligence will be able to learn to play and complete all levels of Geometry dash, using reinforcement learning. To complete this:
1. Connect python with the game
2. Define bounties and penalizations
3. Use Reinforcement learning like Q-learning, Deep Q-learning or Proximal Policy Optimization
4. Build a neuronal nerwork CNN? 
6. Training
7. Eval and tunning

## 1.Connect Python with the Game
---
This step will require:
1. Keyboard Emulation and Automation
- Description: Simulates keystrokes to control the game.
- Tools: Libraries such as pyautogui in Python can be used to simulate keystrokes.
- Challenges: Can be slow and not always accurate.
2. Use of Computer Vision Tools
- Description: Uses computer vision to "see" the state of the game and makes decisions based on that.
- Tools: Libraries such as OpenCV to process the visual output of the game.
- Challenges: Requires image processing and can be computationally intensive.
3. Creating a Custom Game Environment
- Description: Create a simplified version of Geometry Dash that you can fully control.
- Tools: Programming languages such as Python or JavaScript to create a basic version of the game.
- Challenges: Requires time and effort to replicate the game mechanics.

In [ ]:
import pyautogui

# give time to join the game (swap to 5 when it is going to be used to play)
def play_geometry_dash(time):
    time.sleep(5)

    # infinite loop to use the click
    while True:
        pyautogui.click()
        time.sleep(1)
        break

Capture the game

In [ ]:
import cv2
import numpy as np
import pyautogui

# take a screenshot
def capture_screen(region=None):
    # if region is None, will take the whole screen
    screen = pyautogui.screenshot(region=region)
    frame = np.array(screen)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame

def start_capturing_game():
    while True:
        # Define the region of the game
        region = (0, 0, 1920, 1080)  # Reemplaza con las coordenadas y dimensiones correctas
        screen = capture_screen(region)
        # Add here in the future the logic of screen analisis and neuronal netwrok

        # Visualize the screenshot
        cv2.imshow("Game Capture", screen)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()